In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#랭체인 방식
from langchain_openai import ChatOpenAI

query = "지금 미국의 대통령은 누구인가요?"
llm = ChatOpenAI(model='gpt-4o-mini')
llm.invoke(query)

ModuleNotFoundError: No module named 'langchain_openai'

In [3]:
# state
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

class AgentState(TypedDict):
    messages: list[Annotated[AnyMessage,add_messages]]

In [4]:
# graph builder
from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)

In [5]:
def generate(state: AgentState) -> AgentState:
    messages = state['messages']
    ai_messages = llm.invoke(messages)
    return {'messages':[ai_messages]}

In [6]:
from langgraph import graph
from langgraph.graph import START,END
graph_builder.add_node('generate',generate)
graph_builder.add_edge(START, 'generate')
graph_builder.add_edge('generate', END)
graph = graph_builder.compile()

In [7]:
from langchain_core.messages import HumanMessage
initial_state = {'messages':[HumanMessage(content=query)]}
graph.invoke(initial_state)

{'messages': [AIMessage(content='현재 미국의 대통령은 조 바이든(Joe Biden)입니다. 그는 2021년 1월 20일에 취임하였습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 17, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-Ckn6YGZ9qtmxuzA1vxNIrZILJnaCh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b022a-3e8c-7c80-9241-f88c3c765ff3-0', usage_metadata={'input_tokens': 17, 'output_tokens': 31, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}